In [ ]:
import pandas as pd
import nltk
import numpy as np
import re
import math
from scipy import sparse
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from xgboost import XGBClassifier

from gensim.models import KeyedVectors

In [ ]:
file_1 = pd.read_csv("D:/NU/Data Mining/Mini Project/Training_Data/twitter-2013train.txt", sep='\t', header=None)
file_2 = pd.read_csv("D:/NU/Data Mining/Mini Project/Training_Data/twitter-2015train.txt", sep='\t', header=None)
file_3 = pd.read_csv("D:/NU/Data Mining/Mini Project/Training_Data/twitter-2016train.txt", sep='\t', header=None)
t_test = pd.read_csv("D:/NU/Data Mining/Mini Project/new_english_test.csv", sep=',')
Stop_Words = pd.read_csv("D:/NU/Data Mining/Mini Project/stopwords.txt",header=None)
all_files = pd.concat([file_1, file_2, file_3], ignore_index=True)
All_Data = all_files.drop(all_files.columns[[0]], axis=1)
All_Data_2 = All_Data.drop(all_files.columns[[1]], axis=1)
Stop_Words = Stop_Words.iloc[3:]
Stop_Words.columns = ['Words']
StopWords = Stop_Words['Words'].tolist()
All_Data_2.columns = ['Tweet']
test = t_test.drop(t_test.columns[[0]], axis=1)
test.columns = ['Tweet']
for a in StopWords:
    if isinstance(a, str):
        text = a
        decoded = False
    else:
        text = a.decode(encoding)
        decoded = True

In [ ]:
def PreProcess(line):
    line = line.lower()
    curr = ""
    for word in line.split():
        word = re.sub(r'\\[u][0-9]{1,}', '', word)
        word = re.sub(r'@[a-zA-Z0-9_.-]{1,}','',word)
        word = re.sub(r"[^a-zA-Z0-9]+", ' ', word)
        if word not in StopWords:
            curr = curr + word +" "
    curr = curr.strip()
    curr = PorterStemmer().stem(curr)
    curr = curr.strip()
    return curr

def CountEmo_pos(line):
    postive = line.count(':)')
    postive = postive + line.count(':D')
    postive = postive + line.count('=)')
    postive = postive + line.count(':*')
    postive = postive + line.count(':-)')
    postive = postive + line.count('<3')
    return postive

def CountEmo_neg(line):
    negative = line.count(':(')
    negative = negative + line.count(':_(')
    negative = negative + line.count(':''(')
    negative = negative + line.count('=(')
    negative = negative + line.count(':-(')
    negative = negative + line.count(':S')
    return negative

def CountHashtags (line):
    count = line.count('#')
    return count

def CountURLS (line):
    count = line.count('http')
    return count

def CountMentions (line):
    count = line.count('#')
    return count
                                         
def CountExclamationMark(line):
    count = line.count('!')
    return count

In [ ]:
All_Data_2['Positive_Emoticons'] = All_Data_2['Tweet'].apply(CountEmo_pos)
All_Data_2['Negative_Emoticons'] = All_Data_2['Tweet'].apply(CountEmo_neg)
All_Data_2['HashTags'] = All_Data_2['Tweet'].apply(CountHashtags)
All_Data_2['Mentions'] = All_Data_2['Tweet'].apply(CountMentions)
All_Data_2['URLS'] = All_Data_2['Tweet'].apply(CountURLS)
All_Data_2['EM'] = All_Data_2['Tweet'].apply(CountExclamationMark)

In [ ]:
All_Data_2['Tweet'] = All_Data_2['Tweet'].apply(PreProcess)

In [ ]:
train_data,test_data,train_labels,test_labels = train_test_split(All_Data_2,All_Data[1], test_size=0.2)

In [ ]:
t_pos_emotions = pd.DataFrame(np.array(train_data.drop(['Tweet', 'Negative_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
t_neg_emotions = pd.DataFrame(np.array(train_data.drop(['Tweet', 'Positive_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
t_thashtags = pd.DataFrame(np.array(train_data.drop(['Tweet', 'Negative_Emoticons','Positive_Emoticons', 'Mentions', 'URLS', 'EM'],1)))
t_mentions = pd.DataFrame(np.array(train_data.drop(['Tweet', 'HashTags', 'Negative_Emoticons','Positive_Emoticons', 'URLS', 'EM'],1)))
t_urls = pd.DataFrame(np.array(train_data.drop(['Tweet', 'HashTags', 'Mentions', 'Negative_Emoticons','Positive_Emoticons', 'EM'],1)))
t_em = pd.DataFrame(np.array(train_data.drop(['Tweet', 'HashTags', 'Mentions', 'URLS', 'Negative_Emoticons','Positive_Emoticons'],1)))

s_pos_emotions = pd.DataFrame(np.array(test_data.drop(['Tweet', 'Negative_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
s_neg_emotions = pd.DataFrame(np.array(test_data.drop(['Tweet', 'Positive_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
s_thashtags = pd.DataFrame(np.array(test_data.drop(['Tweet', 'Negative_Emoticons','Positive_Emoticons', 'Mentions', 'URLS', 'EM'],1)))
s_mentions = pd.DataFrame(np.array(test_data.drop(['Tweet', 'HashTags', 'Negative_Emoticons','Positive_Emoticons', 'URLS', 'EM'],1)))
s_urls = pd.DataFrame(np.array(test_data.drop(['Tweet', 'HashTags', 'Mentions', 'Negative_Emoticons','Positive_Emoticons', 'EM'],1)))
s_em = pd.DataFrame(np.array(test_data.drop(['Tweet', 'HashTags', 'Mentions', 'URLS', 'Negative_Emoticons','Positive_Emoticons'],1)))

In [ ]:
def train():
    vectorizer = CountVectorizer(min_df=2 ).fit(train_data['Tweet'])
    train_data_vectorized = vectorizer.transform(train_data['Tweet'])
    test_data_vectorized = vectorizer.transform(test_data['Tweet'])

    train_d_vec = pd.DataFrame(train_data_vectorized.todense())
    test_d_vec = pd.DataFrame(test_data_vectorized.todense())

    train_d = pd.concat([train_d_vec, t_pos_emotions], axis=1)
    train_dd = pd.concat([train_d, t_thashtags], axis=1)
    train_ddd = pd.concat([train_dd, t_mentions], axis=1)
    train_dddd = pd.concat([train_ddd, t_urls], axis=1)
    train_ddddd = pd.concat([train_dddd, t_em], axis=1)
    train_dddddd = pd.concat([train_ddddd, t_neg_emotions], axis=1)

    test_d = pd.concat([test_d_vec, s_pos_emotions], axis=1)
    test_dd = pd.concat([test_d, s_thashtags], axis=1)
    test_ddd = pd.concat([test_dd, s_mentions], axis=1)
    test_dddd = pd.concat([test_ddd, s_urls], axis=1)
    test_ddddd = pd.concat([test_dddd, s_em], axis=1)
    test_dddddd = pd.concat([test_ddddd, s_neg_emotions], axis=1)
    
    np.random.seed(0)

    pipe = Pipeline([('classifier', RandomForestClassifier())])

    search_space = [{'classifier': [LogisticRegression()],
                     'classifier__penalty': ['l1', 'l2'],
                     'classifier__C': np.logspace(0, 4, 10)},
                    {'classifier': [RandomForestClassifier()],
                     'classifier__n_estimators': [10, 100, 1000],
                     'classifier__max_features': [1, 2, 3]}]

    clf = GridSearchCV(pipe, search_space, cv=5, verbose=0)
    best_model = clf.fit(train_dddddd, train_labels)

    print (best_model.best_estimator_.get_params()['classifier'])

In [ ]:
train()

In [ ]:
def PreProcess_for_w2v(line):
    line = line.lower()
    curr = ""
    for word in line.split():
        word = re.sub(r'\\[u][0-9]{1,}', '', word)
        word = re.sub(r'@[a-zA-Z0-9_.-]{1,}','',word)
        word = re.sub(r"[^a-zA-Z0-9]+", ' ', word)
        if word not in StopWords:
            curr = curr + word +" "
    return curr

In [ ]:
test['Positive_Emoticons'] = test['Tweet'].apply(CountEmo_pos)
test['Negative_Emoticons'] = test['Tweet'].apply(CountEmo_neg)
test['HashTags'] = test['Tweet'].apply(CountHashtags)
test['Mentions'] = test['Tweet'].apply(CountMentions)
test['URLS'] = test['Tweet'].apply(CountURLS)
test['EM'] = test['Tweet'].apply(CountExclamationMark)

test['Tweet'] = test['Tweet'].apply(PreProcess)

w2v_pre_train = pd.DataFrame(All_Data[2].apply(PreProcess_for_w2v))
w2v_pre_test = pd.DataFrame(test['Tweet'].apply(PreProcess_for_w2v))

w2v_pre_train.columns = ['Tweet']
w2v_pre_test.columns = ['Tweet']

In [ ]:
model = KeyedVectors.load_word2vec_format('D:/NU/Data Mining/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def remove_num(df):
    dd_ff = []
    for i in df:
        ww = ""
        for x in i.split():
            if not (x.isdigit() or x[0] == '-' and x[1:].isdigit()):
                ww = ww + x +" "
        dd_ff.append(ww)
    n_data = pd.DataFrame(dd_ff)
    return dd_ff

In [ ]:
def get_IDF(corpus, xx, yy):
    vectorizer = CountVectorizer()
    cvect = vectorizer.fit_transform(corpus)
    analyze = vectorizer.build_analyzer()
    TF = cvect.toarray()
    
    document_frequency = []
    for i in range(yy):
        r = 0
        for j in range(xx):
            if TF[j][i] >= 1:
                r = r + 1
        document_frequency.append(r)
    
    IDF_2 = []
    for i in document_frequency:
        x = math.log10(xx/i)
        IDF_2.append(x)
    IDF = pd.DataFrame(IDF_2)
    return vectorizer.get_feature_names(), IDF

In [ ]:
n_data = remove_num(w2v_pre_train['Tweet'])
s_data = remove_num(w2v_pre_test['Tweet'])

In [ ]:
train_names, IDF_t = get_IDF(n_data, 16041, 29039)
test_names, IDF_s = get_IDF(s_data, 3035, 9503)

train_names = pd.DataFrame(train_names)
test_names = pd.DataFrame(test_names)

train_idf = pd.concat([train_names, IDF_t], axis=1)
test_idf = pd.concat([test_names, IDF_s], axis=1)

train_idf.columns = ['word', 'idf']
test_idf.columns = ['word', 'idf']

train_t_idf = train_idf.set_index('word')['idf'].to_dict()
test_t_idf = test_idf.set_index('word')['idf'].to_dict()

In [ ]:
def word_2_vec(df, ff):
    ww = pd.DataFrame()
    for i in df:
        w2v2 = pd.DataFrame()
        for x in i.split():
            if not (x.isdigit() or x[0] == '-' and x[1:].isdigit()):
                try:
                    w2v = pd.DataFrame(model[x]*(ff['%s'%x]))
                    w2v2 = pd.concat([w2v2, w2v], axis=1)
                except:
                    x
        w2v2['SUM'] = w2v2.sum(axis=1)
        www = pd.concat([ww, w2v2['SUM']], axis=1)
        ww = www
    word2v = ww.T.reset_index()
    word2vec = word2v.drop(word2v.columns[[0]],axis=1)
    return word2vec

In [ ]:
word_embadding_train = word_2_vec(w2v_pre_train['Tweet'], train_t_idf)
word_embadding_test = word_2_vec(w2v_pre_test['Tweet'], test_t_idf)

In [ ]:
word_embadding_test.info()

In [ ]:
tt_pos_emotions = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'Negative_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
tt_neg_emotions = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'Positive_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
tt_thashtags = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'Negative_Emoticons','Positive_Emoticons', 'Mentions', 'URLS', 'EM'],1)))
tt_mentions = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'HashTags', 'Negative_Emoticons','Positive_Emoticons', 'URLS', 'EM'],1)))
tt_urls = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'HashTags', 'Mentions', 'Negative_Emoticons','Positive_Emoticons', 'EM'],1)))
tt_em = pd.DataFrame(np.array(All_Data_2.drop(['Tweet', 'HashTags', 'Mentions', 'URLS', 'Negative_Emoticons','Positive_Emoticons'],1)))

ss_pos_emotions = pd.DataFrame(np.array(test.drop(['Tweet', 'Negative_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
ss_neg_emotions = pd.DataFrame(np.array(test.drop(['Tweet', 'Positive_Emoticons','HashTags', 'Mentions', 'URLS', 'EM'],1)))
ss_thashtags = pd.DataFrame(np.array(test.drop(['Tweet', 'Negative_Emoticons','Positive_Emoticons', 'Mentions', 'URLS', 'EM'],1)))
ss_mentions = pd.DataFrame(np.array(test.drop(['Tweet', 'HashTags', 'Negative_Emoticons','Positive_Emoticons', 'URLS', 'EM'],1)))
ss_urls = pd.DataFrame(np.array(test.drop(['Tweet', 'HashTags', 'Mentions', 'Negative_Emoticons','Positive_Emoticons', 'EM'],1)))
ss_em = pd.DataFrame(np.array(test.drop(['Tweet', 'HashTags', 'Mentions', 'URLS', 'Negative_Emoticons','Positive_Emoticons'],1)))

In [ ]:
vectorizer = CountVectorizer(min_df=2 ).fit(All_Data_2['Tweet'])
train_data_vectorized = vectorizer.transform(All_Data_2['Tweet'])
test_data_vectorized = vectorizer.transform(test['Tweet'])
train_labels = All_Data[1]

train_d_vec = pd.DataFrame(train_data_vectorized.todense())
test_d_vec = pd.DataFrame(test_data_vectorized.todense())

train_d = pd.concat([train_d_vec, tt_pos_emotions], axis=1)
train_dd = pd.concat([train_d, tt_thashtags], axis=1)
train_ddd = pd.concat([train_dd, tt_mentions], axis=1)
train_dddd = pd.concat([train_ddd, tt_urls], axis=1)
train_ddddd = pd.concat([train_dddd, tt_em], axis=1)
train_dddddd = pd.concat([train_ddddd, tt_neg_emotions], axis=1)
tt_dd = pd.concat([train_dddddd, word_embadding_train], axis=1)

test_d = pd.concat([test_d_vec, ss_pos_emotions], axis=1)
test_dd = pd.concat([test_d, ss_thashtags], axis=1)
test_ddd = pd.concat([test_dd, ss_mentions], axis=1)
test_dddd = pd.concat([test_ddd, ss_urls], axis=1)
test_ddddd = pd.concat([test_dddd, ss_em], axis=1)
test_dddddd = pd.concat([test_ddddd, ss_neg_emotions], axis=1)
ss_dd = pd.concat([test_dddddd, word_embadding_test], axis=1)

In [ ]:
clfr = LogisticRegression()
clfr.fit(tt_dd, train_labels)
predicted = clfr.predict(ss_dd)

In [ ]:
result_labels = pd.DataFrame(predicted)
result_labels.columns = ['sentiment']
result_id = test
Results = pd.concat([t_test['id'], result_labels], axis=1)

In [ ]:
Results.to_csv(path_or_buf="English_Tweets_Results1", columns=['id', 'sentiment'], index=False)